# Hypothesis: Can we predict stock prices using reddit or twitter data?

## Abstract

Predicting stock market movements still remains an open research topic. With the rise in the usage of social media, a huge amount of data is generated by people, every second. For researchers, this deluge of data is like a gold mine, which enables them to measure sentiments of the crowd. Since anyone on social media can contribute to the discussion of stocks of a company, the overall quality of the data might be affected. We will test the hypothesis that whether the discussions on Reddit impact the future stock price. We deployed 3 different models to test our hypothesis. We further concluded that the Reddit discussions are tad-bit influential and have little impact on the future stock prices.

## Introduction

Is stock price still a game of chance? Several factors including economic conditions and quarterly performance impact a company’s stock price. But, is there any other factor which might affect it significantly like social media? 

Twitter and Reddit are not very different. We can identify some situations where the tweets have impacted the market. For instance, on March 30, 2015, a tweet from Elon Musk resulted in an increase of Tesla’s capitalization by approximately US$1 billion in just a few minutes.

<img src="http://artitustech.com/wp-content/uploads/images/elon_tweet.PNG">

To give another example, after a tweet from Carl Icahn in 2013, Apple's capitalization increased to more than $10 billion.

<img src="http://artitustech.com/wp-content/uploads/images/carl_icahn.PNG">

Both the problems are very different: post-tweet analysis and creating a profit-generating real-time high-frequency trading system. 

We have identified few problems in creating the latter:

1. Twitter and Reddit consist of a huge number of bots, with Twitter accounting for about [43 Million bots](https://www.cnbc.com/2017/03/10/nearly-48-million-Twitter-accounts-could-be-bots-says-study.html). 
2. Also, not all people that post on social media are well informed about the stock market or invest in stock market. 
3. Even if we are able to identify the most influential profiles, it becomes difficult to identify such kind of sarcasm:

<img src="http://artitustech.com/wp-content/uploads/images/elon_april_fool.PNG">




This sets the ground for us to get started and making the problem intriguing. Having said that, we will test our hypothesis on Google's stock prices and Reddit topics.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/2f/Google_2015_logo.svg"><img width="248" alt="Philippine-stock-market-board" src="https://upload.wikimedia.org/wikipedia/commons/d/d7/Philippine-stock-market-board.jpg"></a>


### Project content

In this project we will cover the following key points:
1. Analyse Google's stock price, by studying the trends and seasonality
2. Use Traditional algorithms (ARIMA) to analyze the Google's stock price
3. Use sentiments and stock prices to predict the prices in conjunction with RNNs, precisely LSTMs
4. Apply word vectorizers on Reddit titles and see how they influence the change in stock prices, again using LSTM
5. Lastly, we would test the validity of our hypothesis and conclude based on our research and results from the models

We'll be using past Google stock prices from the API provided by [Alphavantage](https://www.alphavantage.co/). We will use [Archive.org](http://archive.org) to get a snapshot of historical Reddit topics and subtopics. Further, we will use NLTK's Sentiment Analyser to get the sentiments.

Table of contents:
- [Abstract](#Abstract)
- [Introduction](#Introduction)
- [Installing the libraries](#Installing-the-libraries)
- [Data Collection](#Data-Collection)
- [Analysis using Traditional algorithms](#Analysis-using-Traditional-algorithms)
- [Using an RNN network with LSTM](#Using-an-RNN-network-with-LSTM)
- [LSTM Using word vectorizer](#LSTM-Using-word-vectorizer)
- [Conclusion](#Conclusion)

## Installing the libraries

Before getting started, you'll need to install the various libraries that we will use. You will need Keras with Tensorflow backend. We will be using [Plotly](https://plot.ly/) for interactive visualizations.

`$ pip install --ignore-installed --upgrade tensorflow` <br>
`$ pip install keras`<br>
   `$ pip install plotly`<br>
   `$ pip install tflearn`

If you want to install everything in a condas virtual environment, follow the steps [here](https://www.tensorflow.org/install/)

In [1]:
#Imports

#Turning off the warnings before exporting the report
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from statsmodels.tsa.stattools import adfuller

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

import tflearn
from tflearn.data_utils import to_categorical, pad_sequences,VocabularyProcessor

import numpy as np
import pandas as pd
import requests
import json
import datetime
import requests

import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
py.init_notebook_mode(connected=True)


c:\users\akshay\anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


## Data Collection


Now that we've installed and loaded the libraries, let's get the historical stock price of **Google**. Here we will use [Alphavantage](https://www.alphavantage.co/) API to with desired date limits.
- Start Date: March 12, 2018
- End date: April 11, 2016 
### Google Stock price extraction

In [2]:
#Get historical sotck price of Google
key = 'X0Y8MMABUAAJQXVF'
link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey='
r = requests.get(link + key)
result = r.json()
daily_price = result["Time Series (Daily)"]
end_year = 2018
end_month = 3
end_day = 12

#Extract only the required dates
end_date = datetime.date(end_year, end_month, end_day)
start_date = end_date - datetime.timedelta(days=700)
date_keys = sorted(daily_price.keys())
date_keys = date_keys[date_keys.index(str(start_date)):].copy()
dates = []
prices = []
for date in date_keys:
    dates.append(date)
    price = daily_price[date]
    prices.append(price['4. close'])
dataset = pd.DataFrame(np.column_stack([dates, prices]), columns=['tmpstamp', 'price'])
dataset['price'] = pd.to_numeric(dataset['price'])
df_price =dataset

### Historic Reddit title extraction and sentiment generation
Here we first generate date string and pass it through the url: <br>
`http://archive.org/wayback/available?url=reddit.com/r/google&timestamp= + date` <br>
This would return a json string with the url where the archive page recides. 

> {"url": "reddit.com/r/google", "timestamp": "10/10/2016", "archived_snapshots": {"closest": {"status": "200", "available": true, "url": "http://web.archive.org/web/20180423233516/https://www.reddit.com/r/google/", "timestamp": "20180423233516"}}}

We then scrape for titles using BeautifulSoup. The required titles are in `{"class": "title may-blank "}`. 
<br>
Once the titles are collected we pass it  through NLTK sentiment analyser. This returns a sentiment score for the titles:

> {'neu': 0.88, 'compound': -0.2197, 'pos': 0.055, 'neg': 0.065}

Here we only use `pos`, the positive sentiments from the above variables. 

**Note**: Remove the comment from `compute()` to run the following code. I have commented it out because this chunk takes 10 minutes to gather all the titles due to the response time of the API and the number of requests. I have ran the code chunk below to generate the required file `redditSentiments.csv`


In [3]:
def get_sentiment_score(sia, year, month, day):
    
    month_str = str(month)
    if len(month_str) < 2:
        month_str = "0" + month_str
        
    day_str = str(day)
    if len(day_str) < 2:
        day_str = "0" + day_str
    date = str(year) + month_str + day_str
    
    url = "http://archive.org/wayback/available?url=reddit.com/r/google&timestamp=" + date
    req_arc = requests.get(url)
    
    #Get required link from the json
    if(req_arc.status_code == 200):
        data = json.loads(req_arc.text)
        red_archive_url = data['archived_snapshots']['closest']['url']
    else:
        red_archive_url = None
        print("Error return code: "+str(req_arc.status_code))
        return (None,None)

    req_arc_page = requests.get(red_archive_url)
    titles=[]
    
    #Use the above link to go to the archieved reddit posts
    if(req_arc_page.status_code == 200):
        soup = BeautifulSoup(req_arc_page.text, 'html.parser')
        all_a = soup.findAll("a", {"class": "title may-blank "})
        for a in all_a:
            titles.extend(a)
        titles_str = ". ".join(titles)
        res = sia.polarity_scores(titles_str)
        return (res["pos"],res["neg"])
    
    else:
        print("Error return code: "+str(req_arc_page.status_code))
        return (None,None)
    
    #Compute sentiments of the titles
def compute():
    sia = SIA()
    date = datetime.date(end_year, end_month, end_day)
    column_names = ['tmpstamp','pos','neg']
    df = pd.DataFrame( columns=column_names)
    for i in range(700):
        stamp = date.year*10000+date.month*100+date.day
        value = get_sentiment_score(sia, date.year, date.month, date.day)
        date = date - datetime.timedelta(days=1)

        new_df = pd.DataFrame([[date,value[0],value[1]]], columns=column_names)
        df = df.append(new_df, ignore_index=True)
    df.to_csv("redditSentiments.csv", sep=',')

# compute()

The chunk below is similar to the above code. The only difference is that we are using it to extract only the titles of the Reddit posts and save it in "redditSentimentsTitle.csv".

In [4]:
def get_title(year, month, day):
    month_str = str(month)
    if len(month_str) < 2:
        month_str = "0" + month_str
    day_str = str(day)
    if len(day_str) < 2:
        day_str = "0" + day_str
    date = str(year) + month_str + day_str
    
    url = "http://archive.org/wayback/available?url=reddit.com/r/google&timestamp=" + date
    req_arc = requests.get(url)
    
    if(req_arc.status_code == 200):
        data = json.loads(req_arc.text)
        if data['archived_snapshots'] == {}:
            return None
        else:
            red_archive_url = data['archived_snapshots']['closest']['url']
    else:
        red_archive_url = None
        print("Error return code: "+str(req_arc.status_code))
        return (None)

    req_arc_page = requests.get(red_archive_url)
    titles=[]

    if(req_arc_page.status_code == 200):
        soup = BeautifulSoup(req_arc_page.text, 'html.parser')
        all_a = soup.findAll("a", {"class": "title may-blank "})
        for a in all_a:
            titles.extend(a)
        titles_str = ". ".join(titles)
        return (titles_str)
    
    else:
        print("Error return code: "+str(req_arc_page.status_code))
        return (None)
def compute():
    date = datetime.date(end_year, end_month, end_day)
    column_names = ['tmpstamp','title']
    df = pd.DataFrame( columns=column_names)
    for i in range(700):
        stamp = date.year*10000+date.month*100+date.day
        value = get_title(date.year, date.month, date.day)
        date = date - datetime.timedelta(days=1)
        new_df = pd.DataFrame([[date,value]], columns=column_names)
        df = df.append(new_df, ignore_index=True)
    df.to_csv("redditSentimentsTitle.csv", sep=',')

# compute()

### Merge datasets and visualize 

We have two data sets `df_sentiments` and `df_price`. We can combine these into one dataframe to carry on with our analysis.

In [5]:
df_sentiments = pd.read_csv('redditSentiments.csv')
df_sentiments = df_sentiments[['tmpstamp','pos','neg']]

#Join
df = pd.merge(df_price, df_sentiments)

#Set date as the index
df['tmpstamp'] = pd.to_datetime(df['tmpstamp'], format='%Y-%m-%d')
df=df.set_index('tmpstamp')

btc_price = go.Scatter(x=df.index, y=df['price'], name= 'Price')

py.iplot([btc_price])

## Analysis using Traditional algorithms

### Stationary check

Traditional timeseies models assume, the timeseries to be stationary. It is said to be stationary if statistical properties such as mean and variance are constant over time. Also, it should have an autocovariance that does not depend on time. This can be checked by using:
1. **Rolling Statistics test:** Here we’ll take the average and variance of the 7 days
2. **Dickey-Fuller Test:** This is a statistical test for checking stationarity, where the null hypothesis is that the timeseries is not stationary.<br>
**Note**: We will plot standard deviation instead of variance to keep the unit similar to mean.

In [6]:
def stationarity_stats(timeseries):
    #rolling statistics
    rolmean = timeseries.rolling(window=12,center=False).mean()
    rolstd = timeseries.rolling(window=12,center=False).std()
    
    #Ploting stats
    original_plt = go.Scatter(x=df_price.index, y=timeseries, name= 'Original')
    mean_plt = go.Scatter(x=df_price.index, y=rolmean, name= 'Rolling Mean')
    rol_std_plt = go.Scatter(x=df_price.index, y=rolstd, name= 'Rolling Std')
    data = [original_plt, mean_plt,rol_std_plt]
    py.iplot(data)
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    df_output = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        df_output['Critical Value (%s)'%key] = value
    print (df_output)
    

In [7]:
stationarity_stats(df['price'])

Results of Dickey-Fuller Test:
Test Statistic                   1.519271
p-value                          0.997594
#Lags Used                      10.000000
Number of Observations Used    472.000000
Critical Value (5%)             -2.867683
Critical Value (10%)            -2.570042
Critical Value (1%)             -3.444281
dtype: float64


In the above analysis, we can see from the graph that mean and variation are not constant. Also, the Dickey-Fuller Test shows that the t-stats is more than the critical value. Hence it is not stationary.<br>
<br>

### Stationarise Time Series

The first step would be to estimate trend and seasonality and remove these to get a stationary series so that our models can be used. We can then convert the predictions to the original scale.<br>
We can **remove trend** by taking a log transform as log suppresses higher values.


In [8]:
ts_log = np.log(df['price'])
ts_log_plt = go.Scatter(x=ts_log.index, y=ts_log, name= 'Log of Price')
py.iplot([ts_log_plt])

**Moving average** can be used to average ‘k’ consecutive values. Since there is specific cycle for stock pricing, calculating moving averages won't suffice. So we use a ***weighted moving average***, where recent values are given a higher weight. **Halflife** is used to define the amount of exponential decay. 

In [9]:
expwighted_avg = ts_log.ewm(halflife=12,ignore_na=False,adjust=True,min_periods=0).mean()
expwighted_avg_plt = go.Scatter(x=expwighted_avg.index, y=expwighted_avg, name='expwighted avg')
data = [expwighted_avg_plt, ts_log_plt]
py.iplot(data)


If we subtract the orange line from the blue, we can get rid of the trend. Note that the first 11 values will not be defined since there were no enough values to calculate mean. We will drop these NaNs and perform our test.

In [10]:
moving_avg =ts_log.rolling(window=12,center=False).mean()
ts_log_moving_avg_diff = ts_log - moving_avg
ts_log_moving_avg_diff.dropna(inplace=True)
stationarity_stats(ts_log_moving_avg_diff)

Results of Dickey-Fuller Test:
Test Statistic                -7.989817e+00
p-value                        2.491853e-12
#Lags Used                     4.000000e+00
Number of Observations Used    4.670000e+02
Critical Value (5%)           -2.867749e+00
Critical Value (10%)          -2.570077e+00
Critical Value (1%)           -3.444431e+00
dtype: float64


The time series now have fewer fluctuations in mean and variance. Also, test statistics is smaller than 1% critical value. In our case we were able to achieve a considerable test statistics. In some case, like sales of items, there is a factor of seasonality. For example, people buy more in December. There are other techniques that can  be used here like, Differencing (taking the difference with a particular time lag) or decomposition (modeling both trend and seasonality and removing them from the model).

## Time Series Forecasting


Our timeseries has significant dependence among values. This can be modeled using ARIMA (Auto-Regressive Integrated Moving Averages). It represents a linear equation. The parameter in the model are:
Number of  Auto-Regressive (p): It is the possible lag-window of dependent variable. For example, if p = 10 the predictors will be x(t-1) to x(t-10)

**Number of MA terms (q):** It is the possible lag-window forecast error in prediction equation. For eample if q is 10, the predictors for x(t) will be e(t-1) to e(t-10) where e(i) = moving average at i - actual value.

**Number of differences (d):** It is the number of nonseasonal differences

To determine the value of 'p' and 'q', we use following analysis.

**Autocorrelation Function (ACF)** is the correlation between the  timeseries with a lagged version of itself

**Autocorrelation Function (PACF)** is the correlation between the timeseries with a lagged version of itself after eliminating the variations already explained by the intervening comparisons

The plots below show confidence intervals. They are used to determine 'p' and 'q'. 'p' and  'q' are the values where PACF and ACF crosses the upper confidence interval for the first time respectitvely. (In our case the graphs are a lot overlapped)

In [11]:
#ACF and PACF plots:
ts_log_diff = ts_log - ts_log.shift()
ts_log_diff.dropna(inplace=True)
lag_acf = acf(ts_log_diff, nlags=20)
lag_pacf = pacf(ts_log_diff, nlags=20, method='ols')


trace0 = go.Scatter(y=lag_acf, name = 'Autocorrelation Function')
trace1 = go.Scatter(y=lag_pacf,name = 'Partial Autocorrelation Function')
data = [trace0,trace1]

layout = {
    'xaxis': {
        'range': [0, 20]
    },
    'yaxis': {
        'range': [-0.2, 1]
    },
    'shapes': [
        {
            'type': 'line',
            'x0': 0,
            'y0': -1.96/np.sqrt(len(ts_log_diff)),
            'x1': 20,
            'y1': -1.96/np.sqrt(len(ts_log_diff)),
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 4,
                'dash': 'dashdot',
            },
        },
        {
            'type': 'line',
            'x0': 0,
            'y0': 1.96/np.sqrt(len(ts_log_diff)),
            'x1': 20,
            'y1': 1.96/np.sqrt(len(ts_log_diff)),
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 4,
                'dash': 'dashdot',
            },
        }, 
    ]
}

fig = {
    'data': data,
    'layout': layout,
}
py.iplot(fig, filename='shapes-lines')

## Testing a model

We try different values for p and q within the range of 0 to 2. We get the best results for following parameters.


In [12]:
model = ARIMA(ts_log, order=(2, 1, 2))  
results_ARIMA = model.fit(disp=0)  

print(results_ARIMA.summary())
print ('RSS: ',sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

ts_log_diff = ts_log - ts_log.shift()
ts_log_diff_plt = go.Scatter(x=ts_log_diff.index, y=ts_log_diff, name='ts_log_diff')

fitted_arima_plt = go.Scatter(x=ts_log_diff.index, y=results_ARIMA.fittedvalues, name='results_ARIMA')

                             ARIMA Model Results                              
Dep. Variable:                D.price   No. Observations:                  482
Model:                 ARIMA(2, 1, 2)   Log Likelihood                1463.761
Method:                       css-mle   S.D. of innovations              0.012
Date:                Fri, 11 May 2018   AIC                          -2915.522
Time:                        20:52:16   BIC                          -2890.455
Sample:                    04-12-2016   HQIC                         -2905.670
                         - 03-09-2018                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0010      0.001      1.933      0.054   -1.43e-05       0.002
ar.L1.D.price     0.2548      0.545      0.467      0.640      -0.814       1.323
ar.L2.D.price    -0.8243      0.448     

c:\users\akshay\anaconda3\envs\tensorflow\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [13]:
py.iplot([ts_log_diff_plt,fitted_arima_plt])



## Rescaling it and calculating RMSE

We try different values for p and q within the range of 0 to 2. We get the best results for following parameters.

In [14]:
predictions_difference = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_difference_cumsum = predictions_difference.cumsum()
predictions_ARIMA_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_difference_cumsum,fill_value=0)
predictions_ARIMA_log.head()

tmpstamp
2016-04-11    3.994708
2016-04-12    3.995721
2016-04-13    3.996491
2016-04-14    3.997007
2016-04-15    3.998400
dtype: float64

In [15]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)

predictions_ARIMA_plt = go.Scatter(x=predictions_ARIMA.index, y=predictions_ARIMA, name='predictions_ARIMA')
py.iplot([btc_price,predictions_ARIMA_plt])

In [16]:
print('RMSE: ',np.sqrt(sum((predictions_ARIMA-df['price'])**2)/len(df['price'])))

RMSE:  3.860430771308521


The above RMSE is big. This value can be interpreted as we will be able to predict the price for next day with an error margin of more than `$3.8`. 

### Accuracy for predicting the trend

Here we are checking how accurately are we able to predict the trend by checking if we were able to predict the increase or decrease in the stock price.

In [17]:
print("Accuracy for predicting the trend: ")
print ((np.sign(np.diff(np.asarray(predictions_ARIMA)))==np.sign(np.diff(np.asarray(df['price'])))).sum()/len(df['price']))

Accuracy for predicting the trend: 
0.5693581780538303



-------------------

## Using an RNN network with LSTM

We used the ARIMA model for predicting the stock prices, but the accuracy of our prediction was not significantly better than predicting it stochastically. Now, in order to improve our accuracy, we plan to use a recurrent neural network using an LSTM cell. LSTM networks have already been proven successful with text data for applications such as NLP(Sundermeyer et Al., 2012) and machine translation (Cho et Al., 2014). The LSTM network has memory, which is capable of remembering across long sequences. Since we want our network to remember the values in the past, this proves to be a good step forward in predicting the stock prices. We will train this network with the extracted sentiments and stock price. 

The internal architecture of an LSTM cell is as follows:


<img src ="https://upload.wikimedia.org/wikipedia/commons/5/53/Peephole_Long_Short-Term_Memory.svg">

### Normalise the data set

Looking at the dataset, we find that the data is unnormalized. In order to normalize our data, we first reshape the values to be in the range of -1 and +1. Next, we can now easily use the MinMaxScaler preprocessing class from the scikit-learn library. As a standard practice, it is good to rescale the data and normalize it to be in the range of 0 to 1. Thus, using MinMaxScaler, we transform features by scaling each feature to be in the range of 0 to 1.

In [18]:
price = df['price'].values.reshape(-1,1)
pos = df['pos'].values.reshape(-1,1)
price = price.astype('float32')
pos = pos.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(price)

### Splitting the data

After we model our data and learn the weights on the training set, we need to evaluate our model on the test set. Thus, we use cross-validation to train the model and test it's performance on the validation set, which is used for tuning the parameters. Next, we validate our model on the test set. Thus, we split the data with 60% of the data in the training set, 20% validation set and remaining 20% in the test set.

In [19]:
train, validate, test = np.split(scaled, [int(.6*len(scaled)), int(.8*len(scaled))])
print("Train: ", len(train),"; Validation: ", len(validate), "; Test: ", len(test))

Train:  289 ; Validation:  97 ; Test:  97


### Preparing the data

We need to prepare the data in a form that can be easily fed to the neural network. We define a new function called create_dataset. The function takes following arguments:
1. the dataset itself, which is a Numpy array that we want to convert in a curated dataset
2. look_back, which is the number of previous time steps to use as input variables to predict the next time period
3. pos, which is a list of positive sentiments

Thus, this method creates a window of features for our model gives us the dataset in a format we would want to feed in the neural network


In [20]:
# https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
def create_dataset(dataset, look_back, pos, sent=False):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        if i >= look_back:
            a = dataset[i-look_back:i+1, 0]
            a = a.tolist()
            if(sent==True):
                a.append(pos[i].tolist()[0])
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
    #print(len(dataY))
    return np.array(dataX), np.array(dataY)

#### Note:

I tested for 5 values of `look_back`. The best result I got was for `look_back = 2`. The LSTM network expects the input data (X) to be provided with a specific array structure in the form of: [samples, time steps, features].

Currently, our data is in the form: [samples, features] and we are framing the problem as one time step for each sample. We can transform the prepared train and test input data into the expected structure.



In [21]:
look_back = 2
trainX, trainY = create_dataset(train, look_back, pos[0:len(train)],sent=True)
validX, validY = create_dataset(validate, look_back, pos[len(train):(len(train)+len(validate))],sent=True)
testX, testY = create_dataset(test, look_back, pos[(len(train)+len(validate)):len(scaled)], sent=True)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
validX = np.reshape(validX, (validX.shape[0], 1, validX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

###  Network Architecture

#### Input layer
We used an LSTM cell as an input layer. It takes an 'input_shape' as an argument which is a tuple of time steps and a number of features. The layer assumes 1 or more input samples and the output dimension for our network is set to 10

#### LSTM layer- hidden layer
The output of the input layer is fed as input to another the LSTM Cell. A traditional LSTM cell with 10 units, a hyperbolic tangent activation function, and a hard sigmoid recurrent activation function.

#### Output layer
The output layer is a dense layer which gives a prediction of the stock prices, based on the weights it has learned through the LSTM network.

#### Hyperparameters, loss function and optimizer
The loss function that has been used to train the network is a mean absolute error loss function since it is not much affected by outliers. The network has been optimized by the Adam optimization function (Kingma et Al., 2014). We are training the network for 500 epochs with a batch size of 32.


In [22]:
model = Sequential()
model.add(LSTM(10, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(10))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
history = model.fit(trainX, trainY, epochs=500, batch_size=32, validation_data=(testX, testY), verbose=0, shuffle=False)

Here we predict the price for the test dataset and visualize the predicted and actual.

### Validation and tuning hyperparameters

Using different network architecture we tried to minimize the RMSE and maximize the accuracy.

In [23]:
yhat_valid = model.predict(validX)
yhat_valid_plt = go.Scatter(y=yhat_valid.T[0], name='yhat')
valid_plt = go.Scatter(y=validY, name= 'testY')
data = [yhat_valid_plt, valid_plt]
py.iplot(data)

In [24]:
Yhat_valid_inverse = scaler.inverse_transform(yhat_valid.reshape(-1, 1))
validY_inverse = scaler.inverse_transform(validY.reshape(-1, 1))
rmse = sqrt(mean_squared_error(validY_inverse, Yhat_valid_inverse))
print('Test RMSE: %.3f' % rmse)
print ('Accuracy:',(np.sign(np.diff(Yhat_valid_inverse.T))==np.sign(np.diff(validY_inverse.T))).sum()/len(Yhat_valid_inverse))

Test RMSE: 0.907
Accuracy: 0.5591397849462365


We see that we get an RMSE around `$1`. This is an improvement over the ARIMA model. This also means that we can predict the price of Google's Stock with an error of `$1`. Also, the accuracy is `59%`.

### Prediction on test set

In [25]:
yhat_test = model.predict(testX)
yhat_test_plt = go.Scatter(y=yhat_test.T[0], name='yhat')
test_plt = go.Scatter(y=testY, name= 'testY')
data = [yhat_test_plt, test_plt]
py.iplot(data)

Now we inverse the data point and test for `RMSE`

In [26]:
Yhat_test_inverse = scaler.inverse_transform(yhat_test.reshape(-1, 1))
testY_inverse = scaler.inverse_transform(testY.reshape(-1, 1))
rmse = sqrt(mean_squared_error(testY_inverse, Yhat_test_inverse))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 2.856


We see that we get an RMSE around `$4.8`. This is not much of an improvement over the ARIMA model. This also means that we can predict the price of Google's stock with an error of `$4.8`.

In [27]:
yhat_test_plt = go.Scatter(y=Yhat_test_inverse.reshape(len(Yhat_test_inverse),), name='yhat')
test_plt = go.Scatter(y=testY_inverse.reshape(len(testY_inverse),), name= 'testY')
data = [yhat_test_plt, test_plt]
py.iplot(data)

In [28]:
print ((np.sign(np.diff(Yhat_test_inverse.T))==np.sign(np.diff(testY_inverse.T))).sum()/len(Yhat_test_inverse))

0.4731182795698925


# LSTM Using word vectorizer

### Preparing the data

1. We have created CSV file for the reddit titles and use the price data frame created above.
2. Next, we merge the two data frames
3. Initialize the word vectorizer and create a matrix of Reddit titles
4. Next we create labels for each of titles based on the difference in price observed in consecutive days
5. Assigned a label of '1' for positive change and '0' for negative change
6. We ignore the first row to avoid the Nan values encountered
7. Finally we create our matrix of word vectors and a vector of our labels to fed to the neural network explained below.


In [29]:
# Creating the dataset

#reading the csv file
df_sentimentsTitle = pd.read_csv('redditSentimentsTitle.csv')
# meging the sentiments and price dataframe
df_final = pd.merge(df_sentimentsTitle,df_price, how='inner')

# creating a matrix of reddit titles
wordings = np.array(df_final['title'])

wordings = wordings[1:] # removing the first title
# initializing the vocabulary processor for vectorizing the words in thr reddit title
vocab_proc = VocabularyProcessor(300)

titles_list = [] # initializing a list of all titles

for doc in wordings:
    titles_list.append(str(doc))
# creating a matrix of word vectors   
totalX = np.array((list(vocab_proc.fit_transform(titles_list))))

# preparing the labels for each of the reddit titles based on change in price
price = np.array((df_final['price'].diff()))

price = price[1:] # removing the first row to avoid Nan values
# assigning labels '1' for positve change and '0' otherwise
price[price>=0]=1
price[price<0]=0
totalY = price

# Split into training and testing data
trainX, testX, trainY, testY = train_test_split(totalX, totalY, test_size=0.1)

## LSTM Network Using Word Vectors

We did sentiment analysis based on the Reddit titles, as we believed stocks prices so have some sentiment value or are at least remotely influential. However, we did not witness much improvement in our accuracy. Thus, our next rational approach was to use another LSTM network using word vectors to predict the change in stock price. LSTMs have been proved to do well in NLP. Because the Reddit titles are also text-based, the reasonable decision was to apply this type of network to our type of data. It is a typical classification problem, trying to predict the change in stock price. The neural network outputs a fully connected layer with a sigmoid or softmax activation function. The architecture of the network is shown below:

<img src ='http://artitustech.com/wp-content/uploads/images/architecture.png' ></img>

###  Network

#### Input layer
The Reddit titles are used as input to the network. For the word embeddings, the maximum length of the word matrix is considered and is used to convert the words to the proper index. All other words that were not present are set to an index of zero. We then zero pad the sentences and keep the maximum length of embeddings to be 300.

#### Convolutional and Max Pooling Layer
The word embeddings are fed to a 1D convolutional layer with a kernel size of 3 and 32 filters. We have added an additional convolutional layer to the recurrent model. It is not the usual norm, but the justification for adding it was that convolutions excel at mapping spacial structure in the input data. Thus, we added a convolutional layer before an LSTM cell. The hypothesis is that the latent sentiment features are represented by the filters in this layer. Now since the output of this network is an indicator of how well the stocks are performing, these features can be learned through backpropagation. A kernel size of 3 has been chosen because the surrounding words are then also used for the filters to account for a direct negation of words. A max pooling layer is added after the convolutional layer to make the location of the filter invariant.


#### LSTM Cell
The output of the convolutional filters is used as input for the LSTM Cell. A traditional LSTM cell with 100 units, a hyperbolic tangent activation function, and a hard sigmoid recurrent activation function. A GRU Cell (Chung et Al., 2015) has also been used, however, this did not improve the accuracy.

#### Output layer
A sigmoid activation function has been used for the output layer because the network outputs a vector with a size of 1. Thus, the problem at hand is more like a binary classification problem.

#### Hyperparameters, loss function and optimizer
The loss function that has been used to train the network is a binary cross entropy loss function since it is a binary classification problem. The network has been optimized by the Adam optimization function (Kingma et Al., 2014). The learning rate at the beginning is set to 0.001. We train the model for 5 epochs with a batch size of 64 (since our input data is small) 


In [30]:
# defining the architecture of the neural network
max_review_length = 300
embeddings = np.max(totalX) +1
trainX = sequence.pad_sequences(trainX, maxlen=max_review_length)
testX = sequence.pad_sequences(testX, maxlen=max_review_length)
#creating the model
model2 = Sequential()
model2.add(Embedding(embeddings, 32, input_length= None))
model2.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
# compiling the model with loss function and optimizer
model2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

print(model2.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          279200    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 335,605
Trainable params: 335,605
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
# training the model
model2.fit(trainX, trainY, validation_split=0.02, epochs=10, batch_size=64,verbose = True)

Train on 424 samples, validate on 9 samples
Epoch 1/10
424/424 [==============================] - 2s 4ms/step - loss: 0.6917 - acc: 0.5259 - val_loss: 0.7269 - val_acc: 0.2222
Epoch 2/10
424/424 [==============================] - 1s 3ms/step - loss: 0.6920 - acc: 0.5448 - val_loss: 0.7731 - val_acc: 0.2222
Epoch 3/10
424/424 [==============================] - 1s 3ms/step - loss: 0.6882 - acc: 0.5448 - val_loss: 0.7468 - val_acc: 0.2222
Epoch 4/10
424/424 [==============================] - 1s 3ms/step - loss: 0.6883 - acc: 0.5448 - val_loss: 0.7330 - val_acc: 0.2222
Epoch 5/10
424/424 [==============================] - 1s 3ms/step - loss: 0.6880 - acc: 0.5448 - val_loss: 0.7349 - val_acc: 0.2222
Epoch 6/10
424/424 [==============================] - 1s 3ms/step - loss: 0.6875 - acc: 0.5519 - val_loss: 0.7487 - val_acc: 0.2222
Epoch 7/10
424/424 [==============================] - 1s 3ms/step - loss: 0.6856 - acc: 0.5660 - val_loss: 0.7512 - val_acc: 0.2222
Epoch 8/10
424/424 [============

In [32]:
# Score trained model.
scores = model2.evaluate(testX, testY, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

49/49 [==============================] - 0s 921us/step
Test loss: 0.7031801647069503
Test accuracy: 0.5510204154617933


### Results

We've trained our model as shown above and the network improves it's performance with each epoch on both the training set and validation set. The loss decreases and accuracy increases, converging to final accuracy values of `53%` and `55%` on training and validation set respectively. However, when we validate our model against the test set, we see an improvement in the performance. We get a loss of `68%` with a model accuracy of `57%`.

# Conclusion 

We implemented 3 state-of-art models - ARIMA, word vectorizer and LSTMs to predict the trend and price. However, the accuracy of these models turned out to be only slightly better than chance. We believe that with additional data sources and extensive research we can further improve the performance. But for now, data science can help us in making informed investment decisions. To conclude, to buy/sell or not to buy/sell stocks is still up for a game! 

**Further reading:**

Keras: https://keras.io/getting-started/sequential-model-guide/ <br>
ARIMA: http://people.duke.edu/~rnau/411arim3.htm <br>
Timeseries using ARIMA: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/ <br>

**References:**
https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/ <br>
https://github.com/llSourcell/bitcoin_prediction <br>
https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/ <br>
https://github.com/EmielStoelinga/CCMLWI <br>